In [1]:
!pip install pyransac3d imageio scikit-image

How we capture the data:

1. Prepare two cellphones, named `cam` and `light`, both equipped with OpenCamera for focus and exposure locking.

2. Mount the `cam` cellphone onto a tripod to capture the ground and object from a stable view.

3. Hold the `light` cellphone in your hands.

4. Turn off the room lights and enable the flashlight on cellphone `light`.

5. Direct the flashlight towards the object to ensure proper illumination, then lock the focus and exposure on both cellphones.

6. Begin capturing images. For each shot, move cellphone `light` to a new position while simultaneously capturing one image from `cam` and one from `light`. Repeat this process multiple times.

7. For the final capture, position the `light` cellphone near the `cam` to align the flashlight and camera view, then capture the last shot.

8. Save the images taken by `cam` into the `camera` directory and the images from `light` into the `light` directory.

9. Turn off the flashlight and restore the room lights.

10. Replace the object with a checkerboard on the ground while keeping the `cam` tripod fixed.

11. Capture the checkerboard from various angles using the `light` cellphone. Save these images into the `light_calibrate` directory.

12. Keep the `cam` fixed and capture the first shot of the checkerboard.

13. Move the `cam` to capture the checkerboard from multiple views. Save these images, along with the image from the previous step, into the `camera_calibrate` directory.

You can download the raw data from https://drive.google.com/file/d/1BTycVSIdzeRY0JmyDeeSxksH7dCXLf47/view?usp=sharing and unzip it to `../raw_data`.

In [2]:
import cv2
import numpy as np
import os
import shutil
from glob import glob
import trimesh
import pyransac3d as pyrsc

`input_path` contains the `camera_calibrate` and `light calibrate` which calibrate the intrinsic matrix of camera views and light views, respectively. We perform intrinsic calibration and pass it to colmap to improve accuracy. 

To capture the calibration images, put a checkerboard on the ground and capture the checkerboard at different angles. Remember the capture the first `camera_calibrate` image at the **same view** when capturing the scene.

In [3]:
input_path = '../raw_data/pink_bear/'

In [4]:
def get_mtx_dist(file_path, nrows, ncols, chess_width=1.0):
    """
    assume nrows >= ncols
    """
    img_paths = sorted(glob(f'{file_path}/*.jpg'))
    imgpoints = []
    objpoints = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    objp = np.zeros((nrows*ncols,3), np.float32)
    objp[:,:2] = np.mgrid[0:nrows,0:ncols].T.reshape(-1,2)
    objp[:, 1] = -objp[:, 1]
    objp *= chess_width # mm
    gray_img = None
    for img_path in img_paths:
        img = cv2.imread(img_path)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        found, corners = cv2.findChessboardCorners(gray_img, (nrows, ncols), None)
        if not found:
            print(f'Warning, cannot detect {img_path}')
            continue
        corners = cv2.cornerSubPix(gray_img, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        objpoints.append(objp)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray_img.shape[::-1], None, None)
    return mtx, dist, gray_img.shape[::-1], rvecs, tvecs

In [5]:
# This function is borrowed from IDR: https://github.com/lioryariv/idr
def load_K_Rt_from_P(filename, P=None):
    if P is None:
        lines = open(filename).read().splitlines()
        if len(lines) == 4:
            lines = lines[1:]
        lines = [[x[0], x[1], x[2], x[3]] for x in (x.split(" ") for x in lines)]
        P = np.asarray(lines).astype(np.float32).squeeze()

    out = cv2.decomposeProjectionMatrix(P)
    K = out[0]
    R = out[1]
    t = out[2]

    K = K / K[2, 2]
    intrinsics = np.eye(4)
    intrinsics[:3, :3] = K

    pose = np.eye(4, dtype=np.float32)
    pose[:3, :3] = R.transpose()
    pose[:3, 3] = (t[:3] / t[3])[:, 0]

    return intrinsics, pose

In [6]:
def save_colmap_intr(file_path, mtx):
    with open(file_path, 'w') as f:
        f.write(f'{mtx[0, 0]:.4f}, {mtx[1, 1]:.4f}, {mtx[0, 2]:.4f}, {mtx[1, 2]:.4f}')

Calibrate the light intrinsic matrix.

In [7]:
mtx, dist, (w, h), _, _ = get_mtx_dist(f'{input_path}/light_calibrate/', 9, 6, 24.25)

In [8]:
np.savetxt(f'{input_path}/light_intr.txt', mtx)
save_colmap_intr(f'{input_path}/light_intr_colmap.txt', mtx)

In [9]:
def undistort_folder(src_file_path, tgt_file_path, mtx, dist):
    os.makedirs(tgt_file_path, exist_ok=True)
    for file_name in sorted(os.listdir(src_file_path)):
        img = cv2.imread(f'{src_file_path}/{file_name}')
        dst = cv2.undistort(img, mtx, dist)
        cv2.imwrite(f'{tgt_file_path}/{file_name}', dst)

Undistort images using the intrinsics matrix so that we can use the simple perspective camera model. Undistorted images stored in `light_undistort`.

In [10]:
for obj in ['pink_bear', 'hamster', 'fries']:
    this_input_path = input_path.replace('pink_bear', obj)
    undistort_folder(f'{this_input_path}/light', f'{this_input_path}/light_undistort', mtx, dist)

The checkerboard has 9x6 patterns, with size 24.25mm.

In [11]:
mtx, dist, (w, h), rvecs, tvecs = get_mtx_dist(f'{input_path}/camera_calibrate/', 9, 6, 24.25)

In [12]:
rmat = cv2.Rodrigues(rvecs[0])[0]

In [13]:
np.savetxt(f'{input_path}/cam_intr.txt', mtx)
save_colmap_intr(f'{input_path}/cam_intr_colmap.txt', mtx)

In [14]:
for obj in ['pink_bear', 'hamster', 'fries']:
    this_input_path = input_path.replace('pink_bear', obj)
    undistort_folder(f'{this_input_path}/camera', f'{this_input_path}/camera_undistort', mtx, dist)

Prepare the colmap input data. Put the last image of `camera_undistort` to `images` because it is a collocated captured camera view. We use it to obtain the camera view pose with respect to other light views.

In [15]:
for obj in ['pink_bear', 'hamster', 'fries']:
    this_input_path = input_path.replace('pink_bear', obj)
    os.makedirs(f'{this_input_path}/colmap_workspace/images', exist_ok=True)
    cur_idx = 0
    for filename in list(sorted(os.listdir(f'{this_input_path}/camera_undistort')))[-1:]:
        shutil.copy(f'{this_input_path}/camera_undistort/{filename}', f'{this_input_path}/colmap_workspace/images/{cur_idx:03d}.jpg')
        cur_idx += 1
    for filename in sorted(os.listdir(f'{this_input_path}/light_undistort')):
        shutil.copy(f'{this_input_path}/light_undistort/{filename}', f'{this_input_path}/colmap_workspace/images/{cur_idx:03d}.jpg')
        cur_idx += 1

Now we should run COLMAP GUI to obtain camera poses and a sparse pointcloud. Remeber to use the intrinsic data in `cam_intr_colmap` and `light_intr_colmap` for `cam` and `light` respectively. The working directory of COLMAP is `colmap_workspace`, where we store the COLMAP results.

In [16]:
input_path = '../raw_data/pink_bear' # change it to 'hamster' and 'fries' and run all cells below to process all the data

These scripts are adapted from `NeuS`, with slight modifications.

In [17]:
!python imgs2poses.py {input_path}/colmap_workspace

../raw_data/pink_bear/colmap_workspace/sparse/0 ['cameras.bin', 'images.bin', 'points3D.bin', 'project.ini']
Don't need to run COLMAP
Post-colmap
Images # 44
Points (31638, 3) Visibility (31638, 44)
Done with imgs2poses


Follow the instructions in https://github.com/Totoro97/NeuS/tree/main/preprocess_custom_data to crop the point cloud of the object, save it as `${input_path}/colmap_workspace/sparse_points_interest.ply`. Additionally, crop the point cloud of the ground and save it as `${input_path}/colmap_workspace/sparse_points_floor.ply`.

In [18]:
!python gen_cameras.py {input_path}/colmap_workspace

Process done!


In [19]:
cameras = np.load(f'{input_path}/colmap_workspace/preprocessed/cameras_sphere.npz')

In [20]:
n_images = len(os.listdir(f'{input_path}/colmap_workspace/images/'))

In [21]:
n_images

44

In [22]:
def get_intr_pose(cameras, i):
    scale_mat = cameras[f'scale_mat_{i}'].astype(np.float32)
    world_mat = cameras[f'world_mat_{i}'].astype(np.float32)
    P = world_mat @ scale_mat
    P = P[:3, :4]
    intr, pose = load_K_Rt_from_P(None, P)
    return intr, pose

In [23]:
cam_scale_mat = cameras['scale_mat_0']

Obtain the ground pose and normalize the coordinate space. One way is to use the first image in the `camera_calibrate` directory to obtain the ground pose relative to the `cam`. Another way is to estimate a ground plane from the sparse COLMAP pointcloud, which is shown here.

`hamster` and `fries` share the same capture environment as `pink_bear`, therefore the calibration is only done once and relevant data is stored in the `pink_bear` directory.

In [24]:
pcl = trimesh.load(f'{input_path}/colmap_workspace/sparse_points_floor.ply')
cam_scale_mat_inv = cameras['scale_mat_inv_0']
pcl.vertices[:] = pcl.vertices[:] @ cam_scale_mat_inv[:3, :3].T + cam_scale_mat_inv[:3, 3]
pcl.vertices[:] = pcl.vertices[:] @ rmat
plane = pyrsc.Plane()
best_eq, _ = plane.fit(pcl.vertices, 0.01)
pcl.export(f'{input_path}/colmap_workspace/sparse_points_normalized.ply');

In [25]:
floor_height = -float(best_eq[-1])/float(best_eq[2])

In [26]:
floor_height

-0.027370467744678672

Visualize the light poses at a point cloud.

In [27]:
intrinsics = []
poses = []
points = []
for i in range(n_images):
    intr, pose = get_intr_pose(cameras, i)
    pose[:3] = rmat.T @ pose[:3]
    points.append(pose[:3, 3])
    intrinsics.append(intr)
    poses.append(pose)
points = np.stack(points)
trimesh.PointCloud(vertices=points).export(f'{input_path}/colmap_workspace/cam_lights.ply');

In [28]:
len(intrinsics), len(poses)

(44, 44)

Draw a mask and save it to `mask/mask.png`. The mask is conservative esimate of which pixels are definitely the ground.

Finally, output training data to `training_data`.

In [29]:
x, y, w, h = 347, 189, 3200, 1800 # crop parameters

In [30]:
output_path = f'{input_path}/training_data'
os.makedirs(f'{output_path}/image', exist_ok=True)
res = {}
cam_intr = intrinsics[0].copy()
cam_intr[0, 2] -= x
cam_intr[1, 2] -= y
res.update({
    'camera_intrinsics': cam_intr,
    'camera_pose': poses[0],
    'light_pose': np.stack(poses[1:]),
    'floor_height': floor_height,
})
for i, img_name in enumerate(sorted(os.listdir(f'{input_path}/camera'))):
    img = cv2.imread(f'{input_path}/camera/{img_name}')
    img = img[y: y+h, x: x+w]
    cv2.imwrite(f'{output_path}/image/{i:03d}.jpg', img)
mask = cv2.imread(f'{input_path}/mask/mask.png')
mask = mask[y: y+h, x: x+w]
cv2.imwrite(f'{output_path}/mask.png', mask)
np.savez(f'{output_path}/params.npz', **res)

In [31]:
params = np.load(f'{input_path}/training_data/params.npz')

In [32]:
list(params.keys())

['camera_intrinsics', 'camera_pose', 'light_pose', 'floor_height']